In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout2d,Dropout
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D,GRU_NCL

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [5]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [6]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [24]:
rnn = Sequential(GRU_NCL(3,4,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.2),
                 GRU_NCL(8,8,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.4),
                 GRU_NCL(16,16,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.5),
                 GRU_NCL(32,32,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.5),
                 GRU_NCL(64,64,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.6))

In [25]:
seq_model = rnn

In [26]:
linear = Sequential(Linear(128,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [27]:
convert = Sequential(Linear(120,60),LeakyReLU(0.1,True),Dropout(0.5))

In [28]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized_NCL(seq_model,linear,convert).to('cuda:0')

In [29]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [30]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [31]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip,patience=0)

epoch:0, train_loss:0.8078112511746006, val_loss:0.9527843594551086
epoch:1, train_loss:0.7474597595207432, val_loss:0.8876733183860779
epoch:2, train_loss:0.7343080858029232, val_loss:0.8981780409812927
Training completed in 207.9835934638977s


In [32]:
model = CNN_RNN2seq_logNormalized_NCL(seq_model,linear,convert).to('cuda:0')

In [33]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [34]:
loss_func = loss_func_generator(SMAPE)

In [35]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.02839584449412, val_loss:38.953269958496094
epoch:1, train_loss:36.867691675492516, val_loss:39.036720275878906
epoch:2, train_loss:36.742372248951014, val_loss:38.83615493774414
epoch:3, train_loss:36.73258246189572, val_loss:38.561012268066406
epoch:4, train_loss:36.65329475402832, val_loss:38.834007263183594
Training completed in 366.2776894569397s
